Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("результаты А_B - cookie_cats 2.csv")

In [3]:
df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [5]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [6]:
df.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

In [12]:
df.groupby('version').agg({'userid': 'count', 'sum_gamerounds': 'sum', 'retention_1': 'mean', 'retention_7': 'mean'})

,userid,sum_gamerounds,retention_1,retention_7
version,,,,
gate_30,44700,2344795,0.448188,0.190201
gate_40,45489,2333530,0.442283,0.182000


На первый взгляд можно предположить что различий между двумя группами нет

In [16]:
control = df[(df['version'] == 'gate_30')] 
test = df[(df['version'] == 'gate_40')] 

In [17]:
n_control = control.shape[0]
n_test = test.shape[0]
ret1_control = control.retention_1.sum()
ret1_test = test.retention_1.sum()
ret7_control = control.retention_7.sum()
ret7_test = test.retention_7.sum()

print(n_control, n_test)
print(ret1_control, ret1_test)
print(ret7_control, ret7_test)

44700 45489
20034 20119
8502 8279


In [18]:
from statsmodels.stats import proportion

In [21]:
chisq1, pvalue1, table1 = proportion.proportions_chisquare(np.array([ret1_control, ret1_test]), np.array([n_control, n_test]))

In [22]:
chisq7, pvalue7, table7 = proportion.proportions_chisquare(np.array([ret7_control, ret7_test]), np.array([n_control, n_test]))

In [23]:
print(f'Результаты для retention_1: chi = {chisq1}, pvalue = {pvalue1}')

Результаты для retention_1: chi = 3.182963657512031, pvalue = 0.07440965529692188


In [25]:
print(f'Результаты для retention_7: chi = {chisq7}, pvalue = {pvalue7}')

Результаты для retention_7: chi = 10.01316732868897, pvalue = 0.0015542499756142788


Для retention_1 статистически значимых различий нет.

Для retention_7 статистически значимые различия присутствуют.

In [26]:
proportion.proportions_ztest(np.array([ret7_control, ret7_test]), np.array([n_control, n_test]))

(3.164358912748191, 0.001554249975614329)

In [27]:
proportion.proportions_ztest(np.array([ret1_control, ret1_test]), np.array([n_control, n_test]))

(1.7840862247974725, 0.07440965529691913)

In [38]:
df.groupby('version')['retention_7'].mean()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Можно сделать вывод о том, что версия gate_30 превосходит gate_40 по среднему retention 7 дней. Следовательно, рекомендуется оставить версию gate_30, так как возвращаемость на 7 день выше.